In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
%matplotlib inline

In [2]:
import nltk

In [7]:
df= pd.read_csv('../Downloads/nlp-getting-started/train.csv')

In [8]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
df.isnull().sum()/len(df)*100

id           0.000000
keyword      0.801261
location    33.272035
text         0.000000
target       0.000000
dtype: float64

In [10]:
df.drop(columns = ['location' , 'keyword'] , inplace = True)

In [11]:
blanks = []

for i,rv in df.iterrows():
    if type(rv['text']) == str:
        if rv['text'].isspace():
            blanks.append(i)

In [12]:
blanks

[]

In [13]:
import string
punc = []

In [14]:
from nltk.corpus import stopwords

In [15]:
def clean_word(text):
    txt = [c for c in text if c not in string.punctuation]
    
    txt = ''.join(txt)
    
    return [word.lower() for word in txt.split() if word not in stopwords.words('english')]
  

In [16]:
df['final_text'] = df['text'].apply(clean_word)

In [17]:
from nltk.stem import WordNetLemmatizer

In [18]:
lemma = WordNetLemmatizer()

In [19]:
def lem(text):
    return ' '.join([lemma.lemmatize(txt) for txt in text])

In [20]:
df['final_text'] = df['final_text'].apply(lem)

In [21]:
df.head()

,id,text,target,final_text
0,1,Our Deeds are the Reason of this #earthquake M...,1,our deed reason earthquake may allah forgive u
1,4,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,All residents asked to 'shelter in place' are ...,1,all resident asked shelter place notified offi...
3,6,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfire evacuation order...
4,7,Just got sent this photo from Ruby #Alaska as ...,1,just got sent photo ruby alaska smoke wildfire...


In [22]:
df.drop(columns = ['text'] , inplace = True)

In [23]:
df.drop(columns = ['id'] , inplace = True)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
from sklearn.pipeline import Pipeline

In [26]:
from sklearn.svm import SVC

In [27]:
s = SVC()

In [28]:
pipe = Pipeline([('vector' , TfidfVectorizer()) , ('s' , SVC())])

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X = df['final_text']

In [31]:
y = df['target'].values

In [32]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=101)

In [33]:
pipe.fit(X_train , y_train)

Pipeline(memory=None,
         steps=[('vector',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('s',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degr

In [34]:
Predictions = pipe.predict(X_test)

In [35]:
from sklearn.metrics import confusion_matrix , classification_report , accuracy_score

In [36]:
print(confusion_matrix(y_test , Predictions))

[[1204  126]
 [ 321  633]]


In [37]:
print(classification_report(y_test , Predictions))

              precision    recall  f1-score   support

           0       0.79      0.91      0.84      1330
           1       0.83      0.66      0.74       954

    accuracy                           0.80      2284
   macro avg       0.81      0.78      0.79      2284
weighted avg       0.81      0.80      0.80      2284

